In [3]:
import torch
import torchvision 
from torchvision import datasets, transforms
from torch import nn
from torch.utils.data import DataLoader, random_split, TensorDataset
from torch.utils.data import Dataset
from collections import OrderedDict
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split

In [4]:
def contar_parametros(model):
    total_parametros = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_parametros

In [5]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [19]:
import os
import pandas as pd
from torchvision.io import read_image

class CustomDataset(Dataset):
    def __init__(self, annotations_file, transform=None, target_transform=None):
        """
        annotations_file: archivo CSV que contiene las características y etiquetas.
        transform: transformación opcional a aplicar a las características.
        target_transform: transformación opcional a aplicar a las etiquetas.
        """
        self.data = pd.read_csv(annotations_file, delim_whitespace=True, header=None)  # Leer datos tabulares
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        # Devuelve el número total de muestras en el dataset
        return len(self.data)

    def __getitem__(self, idx):
        # Obtener una fila del dataset
        row = self.data.iloc[idx]  # Usamos iloc para obtener la fila por posición
        
        # Las características son todos los valores excepto el último (asumiendo que la última columna es la etiqueta)
        features = row.iloc[:-1].values.astype(float)  # Convertir las características a float
        
        # La etiqueta es el último valor
        label = row.iloc[-1]  # Usar iloc para acceder al último elemento (la etiqueta)

        # Aplicar las transformaciones si están definidas
        if self.transform:
            features = self.transform(features)
        if self.target_transform:
            label = self.target_transform(label)
        
        # Convertir las características y etiquetas a tensores
        features = torch.tensor(features, dtype=torch.float32)
        label = torch.tensor(label, dtype=torch.float32)  # O torch.long si son clases categóricas

        return features, label


In [23]:
dataset=CustomDataset(annotations_file="/mnt/d/Repositorios/Deep_Learning/DL_CIC/generalización/data/German_credit/german.data-numeric")
print(len(dataset))

1000


/tmp/ipykernel_26487/2446389708.py:12: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  self.data = pd.read_csv(annotations_file, delim_whitespace=True, header=None)  # Leer datos tabulares


/tmp/ipykernel_26487/3035750618.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label = row[-1]


ValueError: pic should be 2/3 dimensional. Got 1 dimensions.

In [9]:
train_size = int(0.8 * len(dataset))  # 80% para entrenamiento
test_size = len(dataset) - train_size  # 20% para prueba
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Dividir el conjunto de entrenamiento en entrenamiento (90% de 80%) y validación (10% de 80%)
train_size = int(0.9 * len(train_dataset))  # 90% del conjunto de entrenamiento
val_size = len(train_dataset) - train_size  # 10% para validación
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])